In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog

from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models.distillation import TrayColumn
from idaes.generic_models.unit_models.distillation.condenser \
    import CondenserType, TemperatureSpec

In [2]:
from HFCS_emimTf2N_PR import configuration

In [3]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

2021-05-12 09:32:00 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [4]:
m.fs.unit = TrayColumn(default={
                       "number_of_trays": 3,
                       "feed_tray_location":2,
                       "condenser_type": CondenserType.partialCondenser,
                       "condenser_temperature_spec":
                           TemperatureSpec.customTemperature,
                       "property_package": m.fs.props,
                       "has_heat_transfer": False,
                       "has_pressure_change": False})

In [5]:
# Fix Heater H101 inlet conditions
m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.20)
m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.10)
m.fs.unit.feed.mole_frac_comp[0, "emimTf2N"].fix(0.7)

m.fs.unit.feed.flow_mol.fix(1)
m.fs.unit.feed.temperature.fix(301)
m.fs.unit.feed.pressure.fix(1000000)

In [6]:
m.fs.unit.condenser.reflux_ratio.fix(0.25)

m.fs.unit.condenser.condenser_pressure.fix(1000000)

m.fs.unit.reboiler.boilup_ratio.fix(0.35)

m.fs.unit.condenser.vapor_outlet.temperature.fix(280)

In [7]:
# print(degrees_of_freedom(m))
# print(degrees_of_freedom(m.fs.unit.condenser))
# print(degrees_of_freedom(m.fs.unit.reboiler))

In [8]:
# m.fs.unit.feed_tray.initialize() 
m.fs.unit.initialize() 
# m.fs.unit.feed_tray.initialize() 

2021-05-12 09:32:01 [INFO] idaes.init.fs.unit: Begin initialization.
2021-05-12 09:32:01 [WARNING] idaes.init.fs.unit: Solver not provided. Default solver(ipopt) being used for initialization.
2021-05-12 09:32:01 [INFO] idaes.init.fs.unit.feed_tray: Begin initialization.
2021-05-12 09:32:01 [WARNING] idaes.init.fs.unit.feed_tray: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-05-12 09:32:01 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Starting initialization
2021-05-12 09:32:03 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-05-12 09:32:03 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Equilibrium temperature initialization completed.
2021-05-12 09:32:03 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-05-12 09:32:04 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: 

2021-05-12 09:33:06 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-05-12 09:33:06 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Equilibrium temperature initialization completed.
2021-05-12 09:33:07 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-05-12 09:33:08 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-05-12 09:33:08 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Starting initialization
2021-05-12 09:33:09 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-05-12 09:33:09 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Equilibrium temperature initializati

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
# Import python path
import os

# Import idaes model seralizer to store initialized model
from idaes.core.util import model_serializer as ms

In [ ]:
ms.to_json(m, fname="tray.json.gz")


In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
results = solver.solve(m)

status = solver.solve(m, tee = True)

In [ ]:
from idaes.core.util.model_statistics import large_residuals_set

In [ ]:
print(large_residuals_set(m))

In [ ]:
m.display()